In [1]:
!pip install liboqs-python

In [2]:
# kyber_test.py
# Simple demonstration of CRYSTALS-Kyber (Post-Quantum Key Encapsulation Mechanism)
# Using the official reference implementation via the 'liboqs-python' library
# Kyber is a finalist in NIST's Post-Quantum Cryptography standardization (now ML-KEM)

# Install first: pip install liboqs-python
# liboqs-python provides bindings to Open Quantum Safe (OQS) library with Kyber variants

from oqs import KeyEncapsulation
import binascii

def demo_kyber(variant="Kyber512"):  # Options: Kyber512, Kyber768, Kyber1024
    print(f"\n=== CRYSTALS-Kyber {variant} Demo ===\n")
    
    # Create a Kem object for the responder (Bob)
    print("Bob generates keypair...")
    bob = KeyEncapsulation(variant)
    
    # Bob's public key (sent to Alice)
    public_key = bob.generate_keypair()
    print(f"Bob's public key (hex, first 64 chars): {binascii.hexlify(public_key)[:64].decode()}...")
    print(f"Public key length: {len(public_key)} bytes")
    
    # Bob exports his secret key (kept private!)
    secret_key = bob.export_secret_key()
    print(f"Secret key length: {len(secret_key)} bytes\n")
    
    # Alice creates a Kem object with Bob's public key
    print("Alice encapsulates a shared secret using Bob's public key...")
    alice = KeyEncapsulation(variant, public_key)  # Seed optional for determinism
    
    # Alice generates ciphertext and shared secret
    ciphertext, shared_secret_alice = alice.encap_secret()
    print(f"Ciphertext (hex, first 64 chars): {binascii.hexlify(ciphertext)[:64].decode()}...")
    print(f"Ciphertext length: {len(ciphertext)} bytes")
    print(f"Alice's shared secret (hex): {binascii.hexlify(shared_secret_alice).decode()}\n")
    
    # Bob decapsulates the ciphertext to recover the shared secret
    print("Bob decapsulates the ciphertext...")
    shared_secret_bob = bob.decap_secret(ciphertext)
    print(f"Bob's shared secret (hex): {binascii.hexlify(shared_secret_bob).decode()}\n")
    
    # Verify they match
    if shared_secret_alice == shared_secret_bob:
        print("✓ Success! Shared secrets match – secure key exchange complete.")
        print(f"Shared secret length: {len(shared_secret_alice)} bytes")
    else:
        print("✗ Error: Shared secrets do not match!")

# Run demos for all standard security levels
if __name__ == "__main__":
    print("CRYSTALS-Kyber Post-Quantum KEM Demonstration\n")
    print("Security levels:")
    print("- Kyber512  ≈ AES-128")
    print("- Kyber768  ≈ AES-192")
    print("- Kyber1024 ≈ AES-256\n")
    
    demo_kyber("Kyber512")
    demo_kyber("Kyber768")
    demo_kyber("Kyber1024")

liboqs not found, installing it in /Users/jemmeeyung/_oqs
Installing in 5 seconds...
Installing in 4 seconds...
Installing in 3 seconds...
Installing in 2 seconds...
Installing in 1 seconds...


Cloning into 'liboqs'...


Error installing liboqs.
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/site-packages/oqs/oqs.py", line 240, in _load_liboqs
    liboqs = _load_shared_obj(
        name="oqs",
        additional_searching_paths=[oqs_lib_dir, oqs_lib64_dir],
    )
  File "/opt/anaconda3/lib/python3.13/site-packages/oqs/oqs.py", line 139, in _load_shared_obj
    raise RuntimeError(msg)
RuntimeError: No oqs shared libraries found


fatal: Remote branch 0.14.1 not found in upstream origin


SystemExit: 1

/opt/anaconda3/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
